In [1]:
import run

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
import runconfigs # type: ignore
from nna.exp import runutils


In [3]:
def find_upper_taxo(taxo):

    if '.' in taxo:
        taxo_a = taxo.split('.')
    else:
        taxo_a = taxo[:]

    if set(taxo_a)==set('X'):
        return '.'.join(taxo_a)
    if 'X' in taxo_a:
        taxo_a = [x if x!='X' else '0' for x in taxo_a]
    # -1 because we do not change first bit
    for i in range(len(taxo_a)-1):
        if taxo_a[-(i+1)]=='0':
            continue
        else:
            taxo_a[-(i+1)] = '0'
            break
    
    taxo_a='.'.join(taxo_a)
    return taxo_a
def get_root_taxos(org_taxo):
    root_taxos = [ ]
    upper_taxo = find_upper_taxo(org_taxo)
    previous_taxo = org_taxo
    while upper_taxo!=previous_taxo:
        root_taxos.append(upper_taxo)
        previous_taxo = upper_taxo
        upper_taxo = find_upper_taxo(previous_taxo)
    return root_taxos

assert ['1.1.0', '1.0.0']==get_root_taxos('1.1.1')

def create_multi_label_vector(alphabet, y_data):
    # define input string
    # define universe of possible input values
    # alphabet = ['1.1.10','1.1.7']

    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    
    integer_encoded = [ ]
    for point in y_data:
        root_taxos=get_root_taxos(point)
        
        root_taxos.append(point)
#         print(root_taxos)
        int_values=[char_to_int.get(taxo,None) for taxo in root_taxos]
        int_values = [x for x in int_values if x is not None]
        integer_encoded.append(int_values)
    
    onehot_encoded = list()
#     print(integer_encoded)
    for values in integer_encoded:
        letter = [0 for _ in range(len(alphabet))]
        for value in values:
#             print(value)
            letter[value] = 1
        onehot_encoded.append(letter)
    return onehot_encoded
    

In [4]:


def run_exp(wandb_logger_ins):


    config = wandb_logger_ins.config
    loc_per_set = [['12',
                      '14',
                      '27',
                      '49',
                      '31',
                      '39',
                      '44',
                      '45',
                      '48',
                      '19',
                      '16',
                      '21',
                      '38',
                      '41',
                      '20',
                      '29',
                      '37',
                      '15'],
                     ['17', '46', '50', '32', '33', '25', '40'],
                     ['11', '18', '34', '24', '13', '22', '36', '47', '30']]

    target_taxo = ['1.0.0','1.1.0','1.1.10','1.1.7','0.0.0','1.3.0','1.1.8','0.2.0','3.0.0']
    # 0.0.0 anthrophony
    # 0.2.0 aircraft
    # 1.0.0 biophony
    # 1.1.0 bird
    # 1.1.10 songbirds
    # 1.1.7 duck-goose-swan
    # 1.3.0 insect
    # 1.1.8 grouse-ptarmigan
    # 3.0.0 silence

    audio_dataset, _ = run.prepare_dataset()
    audio_dataset = run.dataset_generate_samples(audio_dataset,
                                         runconfigs.EXCERPT_LENGTH)

    other_taxo = set()
    for sound_ins in audio_dataset.values():
        if sound_ins.taxo_code not in target_taxo:
            other_taxo.add(sound_ins.taxo_code)
    other_taxo = list(other_taxo)

    x_data, y_data, location_id_info = run.put_samples_into_array(
            target_taxo, other_taxo, audio_dataset)

    #     onehot_encoded = create_one_hot_vector(target_taxo, y_data)
    multi_label_vector=create_multi_label_vector(target_taxo, y_data)


    X_train, X_test, X_val, y_train, y_test, y_val = run.split_train_test_val(
        x_data, location_id_info, multi_label_vector, loc_per_set)

    model, optimizer, dataloaders, metrics, criterion = run.prepare_run_inputs(
        config, X_train, X_test, X_val, y_train, y_test, y_val)

    print('ready ?')
    checkpoints_dir = runconfigs.EXP_DIR / 'checkpoints'
    checkpoints_dir.mkdir(exist_ok=True)

    runutils.run(model, dataloaders, optimizer, criterion, metrics,
                 config['device'], config, runconfigs.PROJECT_NAME, checkpoints_dir = checkpoints_dir,wandb_logger_ins=wandb_logger_ins)


In [5]:
from ignite.contrib.handlers import wandb_logger


In [6]:
wandb_project_name = runconfigs.PROJECT_NAME
default_config = runconfigs.default_config
config = run.setup_config(default_config, wandb_project_name)

wandb_logger_ins = wandb_logger.WandBLogger(
    project=wandb_project_name,
    # name=runconfigs.PROJECT_NAME,
    config=config,
)

run_exp(wandb_logger_ins)
# return audio_dataset, config 

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: enisberk (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['other-aircraft'] 107.0
['seabirds'] 1.0
['canids'] 1.0
['loons'] 29.0
['other-car'] 37.0
['other-flare'] 11.0
['other-rain'] 20.0
('-> 0 number of samples are deleted because their taxonomy category does not '
 'have enough data.')
-> classes that do not have e

2021-02-20 04:13:08,084 Trainer INFO: Engine run starting with max_epochs=150.


ready ?


/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)
2021-02-20 04:13:41,189 Train Evaluator INFO: Engine run starting wi

...

train loss 0.4799588871449698
val loss 0.47504028104424884
validation roc auc [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 20
train roc auc [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 20


2021-02-20 04:35:46,250 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-02-20 04:36:11,951 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:26
2021-02-20 04:36:11,954 Train Evaluator INFO: Engine run complete. Time taken: 00:00:26
2021-02-20 04:36:11,955 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-02-20 04:36:20,060 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:08
2021-02-20 04:36:20,064 Val Evaluator INFO: Engine run complete. Time taken: 00:00:08
wandb: WARNING Step must only increase in log calls.  Step 21 < 714; dropping {'lr/group_0': 0.001}.
2021-02-20 04:36:20,072 Trainer INFO: Epoch[21] Complete. Time taken: 00:01:04


train loss 0.4737567158884003
val loss 0.4685068045489976
validation roc auc [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 21
train roc auc [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 21


2021-02-20 04:36:50,459 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-02-20 04:37:15,762 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:25
2021-02-20 04:37:15,766 Train Evaluator INFO: Engine run complete. Time taken: 00:00:25
2021-02-20 04:37:15,767 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-02-20 04:37:21,965 Val Evaluator ERROR: Engine run is terminating due to exception: .
2021-02-20 04:37:21,969 Trainer ERROR: Engine run is terminating due to exception: .


KeyboardInterrupt: 